In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from os import walk
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
e1P = "C:/Users/tssan/Desktop/MSE215/11-10/DCIM/102NZ7_2/e1"
e2P = "C:/Users/tssan/Desktop/MSE215/11-10/DCIM/102NZ7_2/e2"
fn1 = next(walk(e1P), (None, None, []))[2]  # [] if no file
fn2 = next(walk(e2P), (None, None, []))[2]  # [] if no file
print(fn1)
print(fn2)


load images: grayscale> viridus colormap

In [ ]:
# Load the two images
img1 = e1P + '/' +fn1[0]
img2 = e2P + '/' +fn2[0]

image1 = cv.imread(img1, cv.IMREAD_GRAYSCALE)
image2 = cv.imread(img2, cv.IMREAD_GRAYSCALE)

plt.imshow(image1)
plt.axis('off')  # Hide the axis
plt.show()

plt.imshow(image2)
plt.axis('off')  # Hide the axis
plt.show()

alignment MKI

In [ ]:

# Check if images are loaded
if image1 is None or image2 is None:
    print("Error: Could not load images. Check the file paths.")
else:
    # Detect ORB keypoints and descriptors
    orb = cv.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Match descriptors using BFMatcher
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract the matched keypoints
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Find the homography matrix
    matrix, mask = cv.findHomography(points1, points2, cv.RANSAC, 5.0)

    # Warp image1 to align with image2
    height, width = image2.shape
    aligned_image = cv.warpPerspective(image1, matrix, (width, height))

    # Display the results using plt.imshow
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Image 1")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Image 2")
    plt.imshow(image2, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Aligned Image 1")
    plt.imshow(aligned_image, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
if image1 is None or image2 is None:
    print("Error: Could not load images. Check the file paths.")
else:
    # Detect keypoints and descriptors using ORB
    orb = cv.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Match features using BFMatcher
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches by distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate affine transformation (scaling + rotation + translation)
    matrix, _ = cv.estimateAffine2D(points2, points1)

    # Apply the transformation to align image2 to image1
    height, width = image1.shape
    aligned_image = cv.warpAffine(image2, matrix, (width, height), flags=cv.INTER_LINEAR)

    # Display the results using matplotlib
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Image 1 (Reference)")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Image 2 (Original)")
    plt.imshow(image2, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Aligned Image 2")
    plt.imshow(aligned_image, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Function to apply Canny edge detection
def apply_canny(image1, image2, param_ranges):
    for low_threshold, high_threshold in param_ranges:
        edges1 = cv.Canny(image1, low_threshold, high_threshold)
        edges2 = cv.Canny(image2, low_threshold, high_threshold)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f"Canny - Image 1 (L:{low_threshold}, H:{high_threshold})")
        plt.imshow(edges1, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Canny - Image 2 (L:{low_threshold}, H:{high_threshold})")
        plt.imshow(edges2, cmap='gray')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Function to apply Sobel filter
def apply_sobel(image1, image2, param_ranges):
    for ksize in param_ranges:
        sobel1_x = cv.Sobel(image1, cv.CV_64F, 1, 0, ksize=ksize)
        sobel1_y = cv.Sobel(image1, cv.CV_64F, 0, 1, ksize=ksize)
        sobel1 = cv.magnitude(sobel1_x, sobel1_y)

        sobel2_x = cv.Sobel(image2, cv.CV_64F, 1, 0, ksize=ksize)
        sobel2_y = cv.Sobel(image2, cv.CV_64F, 0, 1, ksize=ksize)
        sobel2 = cv.magnitude(sobel2_x, sobel2_y)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f"Sobel - Image 1 (ksize:{ksize})")
        plt.imshow(np.uint8(sobel1), cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Sobel - Image 2 (ksize:{ksize})")
        plt.imshow(np.uint8(sobel2), cmap='gray')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Function to apply Laplacian filter
def apply_laplacian(image1, image2, param_ranges):
    for ksize in param_ranges:
        laplacian1 = cv.Laplacian(image1, cv.CV_64F, ksize=ksize)
        laplacian2 = cv.Laplacian(image2, cv.CV_64F, ksize=ksize)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f"Laplacian - Image 1 (ksize:{ksize})")
        plt.imshow(np.uint8(laplacian1), cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Laplacian - Image 2 (ksize:{ksize})")
        plt.imshow(np.uint8(laplacian2), cmap='gray')
        plt.axis('off')

        plt.tight_layout()
        plt.show()


In [ ]:
# Expanded parameter sweeps for each algorithm

if image1 is not None and image2 is not None:
    # Apply Canny edge detection with a wider range of parameters
    canny_params = [(low, high) for low in range(50, 201, 50) for high in range(low + 100, 301, 50)]
    print("Running Canny Edge Detection with parameter sweep...")
    apply_canny(image1, image2, canny_params)

    # Apply Sobel filter with more kernel sizes
    sobel_params = [1, 3, 5, 7, 9]  # Include odd sizes up to 9
    print("Running Sobel Filter with parameter sweep...")
    apply_sobel(image1, image2, sobel_params)

    # Apply Laplacian filter with more kernel sizes
    laplacian_params = [1, 3, 5, 7]  # Include odd sizes up to 7
    print("Running Laplacian Filter with parameter sweep...")
    apply_laplacian(image1, image2, laplacian_params)
else:
    print("Error: Could not load images. Check the file paths.")



In [ ]:
#laplacian of gaussian
def compare_log(image1, image2, ksize=5):
    log1 = cv.Laplacian(cv.GaussianBlur(image1, (ksize, ksize), 0), cv.CV_64F)
    log2 = cv.Laplacian(cv.GaussianBlur(image2, (ksize, ksize), 0), cv.CV_64F)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("LoG - Image 1")
    plt.imshow(log1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("LoG - Image 2")
    plt.imshow(log2, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def compare_histograms(image1, image2):
    hist1 = cv.calcHist([image1], [0], None, [256], [0, 256])
    hist2 = cv.calcHist([image2], [0], None, [256], [0, 256])
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Histogram - Image 1")
    plt.plot(hist1, label="Image 1")
    plt.xlim([0, 256])

    plt.subplot(1, 2, 2)
    plt.title("Histogram - Image 2")
    plt.plot(hist2, label="Image 2", color="orange")
    plt.xlim([0, 256])

    plt.tight_layout()
    plt.show()

def compare_absdiff(image1, image2):
    diff = cv.absdiff(image1, image2)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image 1")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Absolute Difference")
    plt.imshow(diff, cmap='hot')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

from skimage.metrics import structural_similarity as ssim

def compare_ssim(image1, image2):
    score, diff = ssim(image1, image2, full=True)
    diff = (diff * 255).astype("uint8")
    print(f"SSIM: {score}")
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image 1")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Difference (SSIM)")
    plt.imshow(diff, cmap='hot')  # Highlight differences in heatmap
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Expanded parameter sweeps for LoG, SSIM, Absolute Difference, and Histogram Comparison

from skimage.metrics import structural_similarity as ssim

# LoG with parameter sweeps
def sweep_log(image1, image2, ksize_values):
    for ksize in ksize_values:
        log1 = cv.Laplacian(cv.GaussianBlur(image1, (ksize, ksize), 0), cv.CV_64F)
        log2 = cv.Laplacian(cv.GaussianBlur(image2, (ksize, ksize), 0), cv.CV_64F)

        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title(f"LoG - Image 1 (ksize={ksize})")
        plt.imshow(log1, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"LoG - Image 2 (ksize={ksize})")
        plt.imshow(log2, cmap='gray')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# SSIM with no parameter sweeps (SSIM generally doesn't require params)
def compare_ssim(image1, image2):
    score, diff = ssim(image1, image2, full=True)
    diff = (diff * 255).astype("uint8")
    print(f"SSIM: {score}")

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image 1")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Difference (SSIM)")
    plt.imshow(diff, cmap='hot')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Absolute Difference with no parameters
def sweep_absdiff(image1, image2):
    diff = cv.absdiff(image1, image2)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image 1")
    plt.imshow(image1, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Absolute Difference")
    plt.imshow(diff, cmap='hot')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Histogram Comparison
def sweep_histograms(image1, image2):
    hist1 = cv.calcHist([image1], [0], None, [256], [0, 256])
    hist2 = cv.calcHist([image2], [0], None, [256], [0, 256])

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Histogram - Image 1")
    plt.plot(hist1, label="Image 1")
    plt.xlim([0, 256])

    plt.subplot(1, 2, 2)
    plt.title("Histogram - Image 2")
    plt.plot(hist2, label="Image 2", color="orange")
    plt.xlim([0, 256])

    plt.tight_layout()
    plt.show()

# Example usage
if image1 is not None and image2 is not None:
    # LoG parameter sweep
    print("Running LoG with parameter sweep...")
    sweep_log(image1, image2, ksize_values=[3, 5, 7, 9])

    # SSIM comparison
    print("Running SSIM comparison...")
    compare_ssim(image1, image2)

    # Absolute Difference
    print("Running Absolute Difference...")
    sweep_absdiff(image1, image2)

    # Histogram Comparison
    print("Running Histogram Comparison...")
    sweep_histograms(image1, image2)
else:
    print("Error: Could not load images. Check the file paths.")


In [ ]:
import re

def extract_r_number(filename):
    match = re.search(r"r\d{3}", filename)
    return match.group(0) if match else None

# Function to match files based on r#
def match_files_by_r_number(list1, list2):
    matches = []
    for file1 in list1:
        r_number1 = extract_r_number(file1)
        for file2 in list2:
            r_number2 = extract_r_number(file2)
            if r_number1 == r_number2:
                matches.append((file1, file2))
    return matches

# Match the files
matched_files = match_files_by_r_number(fn1, fn2)

# Output the results
print("Matched Files:")
for match in matched_files:
    print(match)

In [ ]:
# Function to calculate SSIM for a matched pair of files
def ssim_sweep(matched_files):
    ssim_scores = {}
    for file1, file2 in matched_files:
        # Load the images in grayscale
        image1 = cv.imread(e1P + "/" + file1, cv.IMREAD_GRAYSCALE)
        image2 = cv.imread(e2P + "/" +file2, cv.IMREAD_GRAYSCALE)
        
        if image1 is None or image2 is None:
            print(f"Error: Could not load one or both files: {file1}, {file2}")
            continue
        
        # Compute SSIM and the difference image
        score, diff = ssim(image1, image2, full=True)
        diff = (diff * 255).astype("uint8")
        ssim_scores[(file1, file2)] = score

        # Display the images and SSIM difference
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title(f"Image 1: {file1}")
        plt.imshow(image1, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title(f"Image 2: {file2}")
        plt.imshow(image2, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title(f"SSIM Difference: {score:.4f}")
        plt.imshow(diff, cmap='hot')
        plt.axis('off')

        plt.tight_layout()
        plt.show()
    
    return ssim_scores



# Run the SSIM sweep
ssim_results = ssim_sweep(matched_files)

# Print the SSIM scores
print("\nSSIM Scores:")
for pair, score in ssim_results.items():
    print(f"{pair[0]} <-> {pair[1]}: {score:.4f}")


# Structural Similarity Index Measure (SSIM)

SSIM evaluates the perceptual similarity between two images by combining **luminance**, **contrast**, and **structure** components.

---

## SSIM Components

1. **Luminance Comparison**:
   $$
   l(x, y) = \frac{2\mu_x \mu_y + C_1}{\mu_x^2 + \mu_y^2 + C_1}
   $$
   - $\mu_x$: Mean of image $x$.
   - $\mu_y$: Mean of image $y$.
   - $C_1$: Stabilizing constant to avoid division by zero.

2. **Contrast Comparison**:
   $$
   c(x, y) = \frac{2\sigma_x \sigma_y + C_2}{\sigma_x^2 + \sigma_y^2 + C_2}
   $$
   - $\sigma_x$: Standard deviation of image $x$.
   - $\sigma_y$: Standard deviation of image $y$.
   - $C_2$: Stabilizing constant.

3. **Structure Comparison**:
   $$
   s(x, y) = \frac{\sigma_{xy} + C_3}{\sigma_x \sigma_y + C_3}
   $$
   - $\sigma_{xy}$: Covariance of $x$ and $y$.
   - $C_3$: Typically $C_3 = C_2 / 2$.

---

## Overall SSIM Formula

The final SSIM score combines the three components:
$$
SSIM(x, y) = \left[ l(x, y) \right]^\alpha \cdot \left[ c(x, y) \right]^\beta \cdot \left[ s(x, y) \right]^\gamma
$$
- $\alpha, \beta, \gamma$: Exponents determining the weight of each component (commonly set to 1).

---

## SSIM Interpretation
- **SSIM Range**: $0 \leq SSIM \leq 1$
  - $SSIM = 1$: The images are identical.
  - $SSIM = 0$: The images are completely different.
